In [1]:
import random

subjects = [
    "deep learning", "machine learning", "artificial intelligence",
    "data science", "neural networks", "computer vision",
    "natural language processing", "big data", "cloud computing"
]

verbs = [
    "is used for", "helps in", "improves", "supports",
    "enables", "optimizes", "transforms"
]

objects = [
    "image recognition", "text analysis", "speech recognition",
    "prediction systems", "automation", "decision making",
    "pattern recognition"
]

extras = [
    "in modern applications",
    "for real world problems",
    "using large datasets",
    "with high accuracy",
    "in production systems",
    "at scale"
]

def generate_sentence():
    return f"{random.choice(subjects)} {random.choice(verbs)} {random.choice(objects)} {random.choice(extras)}"

# Generate BIG dataset
with open("large_corpus.txt", "w", encoding="utf-8") as f:
    for _ in range(100_000):   # 🔥 BIG DATA
        f.write(generate_sentence() + "\n")

print("Large dummy dataset created!")


Large dummy dataset created!


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
tokenizer = Tokenizer(
    num_words=20000,
    oov_token="<OOV>"
)

with open("large_corpus.txt", "r", encoding="utf-8") as f:
    for line in f:
        tokenizer.fit_on_texts([line.lower()])

total_words = len(tokenizer.word_index) + 1
print("Vocab size:", total_words)


Vocab size: 54


In [14]:
max_len = 0
with open("large_corpus.txt", "r", encoding="utf-8") as f:
    for line in f:
        seq = tokenizer.texts_to_sequences([line])[0]
        max_len = max(max_len, len(seq))

print("Max length:", max_len)


Max length: 12


In [5]:
def data_generator(file_path, tokenizer, max_len, batch_size=128):
    sequences = []

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            token_list = tokenizer.texts_to_sequences([line.lower()])[0]

            for i in range(1, len(token_list)):
                sequences.append(token_list[:i+1])

                if len(sequences) == batch_size:
                    padded = pad_sequences(sequences, maxlen=max_len, padding="pre")
                    X = padded[:, :-1]
                    y = padded[:, -1]
                    y = tf.keras.utils.to_categorical(y, num_classes=total_words)
                    yield X, y
                    sequences = []


In [6]:
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator("large_corpus.txt", tokenizer, max_len),
    output_types=(tf.float32, tf.float32),
    output_shapes=((None, max_len-1), (None, total_words))
).prefetch(tf.data.AUTOTUNE)


Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


In [7]:
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 128),
    tf.keras.layers.SimpleRNN(128),
    tf.keras.layers.Dense(total_words, activation="softmax")
])

rnn_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

rnn_model.fit(dataset, epochs=5, steps_per_epoch=1000)


Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.5579 - loss: 1.3597
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.6526 - loss: 0.7796
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.6558 - loss: 0.7759
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.6578 - loss: 0.7756
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.6566 - loss: 0.7770


In [8]:
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 128),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(total_words, activation="softmax")
])

lstm_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

lstm_model.fit(dataset, epochs=8, steps_per_epoch=1000)


Epoch 1/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 77s 76ms/step - accuracy: 0.5087 - loss: 1.5396
Epoch 2/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 74s 74ms/step - accuracy: 0.6521 - loss: 0.7816
Epoch 3/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 74s 74ms/step - accuracy: 0.6550 - loss: 0.7777
Epoch 4/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 83s 83ms/step - accuracy: 0.6565 - loss: 0.7772
Epoch 5/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 82ms/step - accuracy: 0.6559 - loss: 0.7785
Epoch 6/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 64s 64ms/step - accuracy: 0.6559 - loss: 0.7765
Epoch 7/8
   2/1000 ━━━━━━━━━━━━━━━━━━━━ 1:20 81ms/step - accuracy: 0.6621 - loss: 0.7885

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 90s 90ms/step - accuracy: 0.6537 - loss: 0.7766
Epoch 8/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 75s 75ms/step - accuracy: 0.6544 - loss: 0.7752


In [9]:
gru_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 128),
    tf.keras.layers.GRU(256),
    tf.keras.layers.Dense(total_words, activation="softmax")
])

gru_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

gru_model.fit(dataset, epochs=8, steps_per_epoch=1000)


Epoch 1/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 95s 83ms/step - accuracy: 0.5580 - loss: 1.3439
Epoch 2/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 58s 58ms/step - accuracy: 0.6522 - loss: 0.7810
Epoch 3/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - accuracy: 0.6550 - loss: 0.7776
Epoch 4/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - accuracy: 0.6570 - loss: 0.7773
Epoch 5/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 56s 56ms/step - accuracy: 0.6560 - loss: 0.7789
Epoch 6/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 44s 44ms/step - accuracy: 0.6566 - loss: 0.7768
Epoch 7/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - accuracy: 0.6540 - loss: 0.7768
Epoch 8/8
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - accuracy: 0.6539 - loss: 0.7754


In [10]:
def predict_next_word(text, model):
    seq = tokenizer.texts_to_sequences([text.lower()])[0]
    seq = pad_sequences([seq], maxlen=max_len-1, padding="pre")
    pred = np.argmax(model.predict(seq), axis=-1)

    for word, idx in tokenizer.word_index.items():
        if idx == pred:
            return word


In [15]:
print(predict_next_word("deep learning is", lstm_model))
print(predict_next_word("deep learning is", gru_model))
print(predict_next_word("deep learning is", rnn_model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
used
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
used
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
used


In [12]:
def safe_predict(text, model):
    if not text.strip():
        return "Invalid input"

    try:
        seq = tokenizer.texts_to_sequences([text.lower()])[0]
        if len(seq) == 0:
            return "No valid tokens"

        seq = pad_sequences([seq], maxlen=max_len-1, padding="pre")
        pred = np.argmax(model.predict(seq), axis=-1)

        for word, idx in tokenizer.word_index.items():
            if idx == pred:
                return word
    except Exception as e:
        return str(e)


In [16]:
test_cases = [
    "deep learning is",
    "machine learning helps in",
    "neural networks enable",
    "natural language processing is used for",
    "learning",
    "quantum learning is",
    "",
    "@@@@ #### !!!"
]

models = {
    "RNN": rnn_model,
    "LSTM": lstm_model,
    "GRU": gru_model
}

for name, model in models.items():
    print(f"\n--- Testing {name} ---")
    for text in test_cases:
        print(f"Input: '{text}' → Output: {safe_predict(text, model)}")



--- Testing RNN ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Input: 'deep learning is' → Output: used
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Input: 'machine learning helps in' → Output: text
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Input: 'neural networks enable' → Output: text
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Input: 'natural language processing is used for' → Output: automation
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Input: 'learning' → Output: science
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Input: 'quantum learning is' → Output: used
Input: '' → Output: Invalid input
Input: '@@@@ #### !!!' → Output: No valid tokens

--- Testing LSTM ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Input: 'deep learning is' → Output: used
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Input: 'machine learning helps in' → Output: automation
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Input: 'neural networks enable' → Output: prediction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Input: 'natural language processing is used for' → Outpu

In [17]:
import pickle
import json

lstm_model.save("model/lstm_model.h5")
pickle.dump(tokenizer, open("model/tokenizer.pkl", "wb"))

with open("model/config.json", "w") as f:
    json.dump({"max_len": max_len}, f)
